In [1]:
import pandas as pd

from src.utils.coordinate_system import Dimensions
import numpy as np
from src.utils.ROI import ROI

img_dims = Dimensions(2048, 1536)

ROI.WIDTH = ROI.HEIGHT = 64

ROI.N_HORIZONTAL = img_dims.width // ROI.WIDTH
ROI.N_VERTICAL = img_dims.height // ROI.HEIGHT

comparison_range_pixels = 400  # the furthest we compare ROIs to in terms of pixels

rois = np.zeros(shape=(ROI.N_HORIZONTAL, ROI.N_VERTICAL), dtype=ROI)
for x in range(ROI.N_HORIZONTAL):
    for y in range(ROI.N_VERTICAL):
        rois[x, y] = ROI(x, y)

# filtering ROIs
filtering_mask = np.random.rand(ROI.N_HORIZONTAL, ROI.N_VERTICAL) < 0.5

filtered_rois = rois[filtering_mask]
filtered_rois = filtered_rois.flatten()

# filtered_rois = rois.flatten()

# print('Filtered ROIs:\n', filtered_rois, '\n')
# 
# # printing
# print(f"ROI width: {ROI.WIDTH}, ROI height: {ROI.HEIGHT}")
# print(f"N_HORIZONTAL: {ROI.N_HORIZONTAL}, N_VERTICAL: {ROI.N_VERTICAL}")
# print(f"N ROIs: {len(filtered_rois)}")
# # filtered_rois

In [2]:
distances = pd.DataFrame(1, columns=filtered_rois, index=filtered_rois)
# make the diagonal 0
for i in range(len(filtered_rois)):
    distances.iloc[i, i] = 0


# 2nd Approach (within range)

In [72]:
import time
import math

st = time.time()

# Convert the pixel distance into a distance of ROI indexes
# We do this by dividing the range in pixels by the ROI width/height and then rounding it up
x_range = math.ceil(comparison_range_pixels / ROI.WIDTH)
y_range = math.ceil(comparison_range_pixels / ROI.HEIGHT)

# create a dictionary with an entry for each of the filtered ROIs and tells us which other ROIs it should be compared with
comparisons = {}

# construct a new empty matrix for the ROIs
rois_matrix = np.zeros((ROI.N_HORIZONTAL, ROI.N_VERTICAL), dtype=ROI)

# fill it with the filtered ROIs. Leave the rest blank
for roi in filtered_rois:
    rois_matrix[roi.x_idx, roi.y_idx] = roi

# go through each of the filtered ROIs and fill the dictionary by checking which filtered ROIs are within range
rois_matrix_copy = rois_matrix.copy()

for roi in filtered_rois:
    # Calculate the boundaries of the ROI indexes
    x_left = roi.x_idx - x_range
    y_top = roi.y_idx - y_range
    x_right = roi.x_idx + x_range
    y_bottom = roi.y_idx + y_range

    # They need to be bounded between 0 and the maximum index on that axis
    x_left = max(0, x_left)
    y_top = max(0, y_top)
    x_right = min(x_right, ROI.N_HORIZONTAL - 1)
    y_bottom = min(y_bottom, ROI.N_VERTICAL - 1)

    # We remove this ROI from the matrix, so that it's not compared with itself, and so that the ROIs aren't compared in the reverse order as well (only in one order)
    # E.g. if we compare ROI(0;0) to ROI(1;0), we don't want ROI(1;0) to also be compared to ROI(0;0) in that order because that would be an unnecessary comparison.
    rois_matrix_copy[roi.x_idx, roi.y_idx] = 0

    # Select the ROIs within these boundaries
    # We add 1 because the upper boundary is not included when indexing
    within_bounds = rois_matrix_copy[x_left: x_right + 1, y_top: y_bottom + 1]

    # print(roi)
    # print(within_bounds)
    # print()

    # Now we select all the values that are not nan    
    # rois_within_bounds = within_bounds.stack(dropna=True).to_numpy()
    rois_within_bounds = within_bounds[within_bounds != 0]

    # we set this as the dict entry
    if rois_within_bounds.size > 0:
        comparisons[roi] = rois_within_bounds

print(f'Elapsed time: {time.time() - st}s')

In [73]:
rois_matrix

In [67]:
for key, value in comparisons.items():
    print(key)
    for roi in value:
        print(f'{roi}, ', end='')
    if value.size == 0:
        print('-', end='')
    print('\n')

# 1st Approach ("dumb" comparison)

In [59]:
from scipy.spatial import distance


def roi_spatial_distance(roi1: ROI, roi2: ROI):
    """This is our adjacency measure. If the ROIs are too far apart, they can't be from the same cell, and we needn't
    compare them"""
    # get the upper left corners of both ROIs
    ul_1, _ = roi1.coordinates()
    ul_2, _ = roi2.coordinates()

    # Compare the upper left corners. this should be the same distance as if we compared the centers
    return distance.euclidean((ul_1.x, ul_1.y), (ul_2.x, ul_2.y))


In [60]:
import time

# -- Parameters --------------
distance_threshold_rois = 1.5
dist_thresh_pixels = 400
# ----------------------------

st = time.time()

comparisons = {}
rois_copy = list(filtered_rois)
n_comparisons = 0

# for each roi, check if each remaining roi in the list is within its range. 
for roi1 in filtered_rois:
    # delete this roi from rois_copy, so that comparisons are only made in one order of the pairs
    rois_copy.remove(roi1)

    # check which rois are within the distance for this roi
    within_dist = []
    for roi2 in rois_copy:
        spatial_dist = roi_spatial_distance(roi1, roi2)
        n_comparisons += 1

        # print(roi1, roi2, ' : ', f'{spatial_dist:.2f}', ':', spatial_dist < dist_thresh_pixels)
        if spatial_dist < dist_thresh_pixels:
            within_dist.append(roi2)

    comparisons[roi1] = within_dist

print(f"\nCalculating Comparisons: {time.time() - st:.2f}s")

# comparisons
n_comparisons

# Timing
## 1st Approach (comparing whole list)
| Size | n_rois | Duration    | n_comparisons |
|------|--------|-------------|---------------|
| 64   | 768    | 1.03s       | 294.528       |
| 32   | 3072   | 16.61s      | 4.717.056     |
| 16   | 12288  | 260s (est.) | 75.491.328    | 
| 8    | 49152  |             | 1.207.934.976 | 


To calculate n_comparisons with $r$ ROIs, we calculate$\frac{r^2-r}{2}$.  
290.000 comparisons take about 1s. This will probably increase when RAM runs out and memory swap is activated though!

## 2nd Approach (comparing within bounds)
comparison_range_pixels = 400

| Size | n_rois | Duration | n_comparisons |
|------|--------|----------|---------------|
| 64   | 768    | 0.01s    |               |
| 32   | 3072   | 0.05s    |               |
| 16   | 12288  | 0.46s    |               | 
| 8    | 49152  | 6.47s    |               | 

